# 01 Data consistency checks
    
### Importing libraries
### Finding missing data
### Finding duplicates

## Orders data - Consistency checks
### Checking for mixed type fields
### Checking for missing data
### Addressing missing data
### Checking for duplicates

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import os

In [2]:
#Creating root path for data importing
path = r"C:\Users\cavba\OneDrive\Documents\Instacart Basket Analysis"

In [3]:
#Importing data frames original products.csv
df_prods = pd.read_csv(os.path.join(path, "02 Data", "Original Data", "products.csv"), index_col = False)

In [4]:
#Importing orders.csv from Prepared Data
df_ords = pd.read_csv(os.path.join(path, "02 Data", "Prepared Data", "orders_wrangled.csv"), index_col = False)

## Finding missing data

In [5]:
# Checking for missing values
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [6]:
# Creating a subset of df_prods to view missing values
df_nan = df_prods[df_prods['product_name'].isnull()==True]

In [7]:
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [8]:
df_prods.shape

(49693, 5)

In [9]:
df_prods_clean = df_prods[df_prods['product_name'].isnull()==False]

In [10]:
df_prods_clean.shape

(49677, 5)

## Finding duplicates

In [1]:
df_dups = df_prods_clean[df_prods_clean.duplicated()]

NameError: name 'df_prods_clean' is not defined

In [12]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [13]:
df_prods_clean.shape

(49677, 5)

In [14]:
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [15]:
df_prods_clean_no_dups.shape

(49672, 5)

In [16]:
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'))

# df_ords consistency check

## Task 4.5 Step 2

In [51]:
#Describing products data set
df_prods.describe()

,product_id,aisle_id,department_id,prices
count,49693.000000,49693.000000,49693.000000,49693.000000
mean,24844.345139,67.770249,11.728433,9.994136
std,14343.717401,38.316774,5.850282,453.519686
min,1.000000,1.000000,1.000000,1.000000
25%,12423.000000,35.000000,7.000000,4.100000
50%,24845.000000,69.000000,13.000000,7.100000
75%,37265.000000,100.000000,17.000000,11.200000
max,49688.000000,134.000000,21.000000,99999.000000


It looks like the product_id column could have duplicated because the max is lower than the count. The max for the prices column seems unreasonably high compared to the mean and 75th percentile. 

## Task 4.5 Step 3: Checking for mixed type data

In [41]:
# Checking for mixed-type data in df_ords
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

There are no mixed-type columns in the df_ords dataframe

## Task 4.5 Step 5: Checking for missing data

In [43]:
# Checking for missing values in df_ords dataframe
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
order_number                   0
order_days_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
ordered_today                  0
dtype: int64

There are a lot of missing values in the days_since_prior_order column. This may not be a missing data issue considering some customers might place orders often. 

## Task 4.5 Step 6: Addressing missing values

In [26]:
#Adding a true or false column for ordered_today
df_ords['ordered_today'] = np.where(df_ords['days_since_prior_order'].isnull(), True, False)

In [27]:
df_ords.head()

,Unnamed: 0,order_id,user_id,order_number,order_days_of_week,order_hour_of_day,days_since_prior_order,ordered_today
0,0,2539329,1,1,2,8,NaN,True
1,1,2398795,1,2,3,7,15.0,False
2,2,473747,1,3,3,12,21.0,False
3,3,2254736,1,4,4,7,29.0,False
4,4,431534,1,5,4,15,28.0,False


In [28]:
df_ords.shape

(3421083, 8)

In [29]:
# Creating a subset where days_since_prior_order is null
df_zero_days_since = df_ords[df_ords['days_since_prior_order'].isnull()==True]

In [30]:
df_zero_days_since.shape

(206209, 8)

Keeping this information null is important because this subset could be used to compare trends for frequent instacart shoppers

In [31]:
#Exporting df_zero_days_since to Prepared Data folder to save for future analysis
df_zero_days_since.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_recent_checked.csv'))

## Task 4.5 Step 8: Checking for duplicates

In [44]:
# Checking df_ords for duplicate values
df_ords_dups=df_ords[df_ords.duplicated()]


In [45]:
df_ords_dups.head()

,Unnamed: 0,order_id,user_id,order_number,order_days_of_week,order_hour_of_day,days_since_prior_order,ordered_today


In [46]:
df_ords_dups.shape

(0, 8)

There are no duplicates in the df_ords dataframe.

In [47]:
df_ords_clean = df_ords

In [48]:
df_ords_clean.head()

,Unnamed: 0,order_id,user_id,order_number,order_days_of_week,order_hour_of_day,days_since_prior_order,ordered_today
0,0,2539329,1,1,2,8,NaN,True
1,1,2398795,1,2,3,7,15.0,False
2,2,473747,1,3,3,12,21.0,False
3,3,2254736,1,4,4,7,29.0,False
4,4,431534,1,5,4,15,28.0,False


In [49]:
#Exporting clean df_ords dataframe to Prepared Data folder
df_ords_clean.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_cleaned.csv'))